# Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [3]:
class ImageQualityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        quality = self.data.iloc[idx, 0]  # Quality column
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([quality, authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [4]:
class QualityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Predict quality and realness
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

## Setup section

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = 'Dataset/AIGCIQA2023/mos_data.csv'

# Create the dataset
dataset = ImageQualityDataset(csv_file=annotations_file, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 64
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}



# Models loading

In [6]:
NOISY_PRUNED_MODEL_PATH = 'Models/noise_out_pruned_model.pth'
NEGATIVE_IMPACT_PRUNED_MODEL_PATH = 'Models/negative_impact_pruned_model.pth'

noisy_pruned_model = QualityPredictor()
noisy_pruned_model.load_state_dict(torch.load(NOISY_PRUNED_MODEL_PATH, weights_only=True))

negative_impact_pruned_model = QualityPredictor()
negative_impact_pruned_model.load_state_dict(torch.load(NEGATIVE_IMPACT_PRUNED_MODEL_PATH,weights_only=True))

baseline_model = QualityPredictor()
baseline_model.load_state_dict(torch.load('Models/VGG-16_finetuned_regression.pth',weights_only=True))

<All keys matched successfully>

In [11]:
# get an item from the dataset
image, labels = dataset[0]
image = image.unsqueeze(0)
labels = labels.unsqueeze(0)

# predicting labels using the models
noisy_pruned_model.eval()
negative_impact_pruned_model.eval()
baseline_model.eval()

baseline_predictions, baseline_features = baseline_model(image)
noisy_pruned_predictions, noisy_features = noisy_pruned_model(image)
negative_impact_pruned_predictions, neg_impact_features = negative_impact_pruned_model(image)

print("Baseline Model Predictions: ", baseline_predictions)
print("Noisy Pruned Model Predictions: ", noisy_pruned_predictions)
print("Negative Impact Pruned Model Predictions: ", negative_impact_pruned_predictions)

print("Baseline Model Features shape ", baseline_features.shape)
print("Noisy Pruned Model Features shape ", noisy_features.shape)
print("Negative Impact Pruned Model Features shape ", neg_impact_features.shape)




Baseline Model Predictions:  tensor([[42.4086, 40.0079]], grad_fn=<AddmmBackward0>)
Noisy Pruned Model Predictions:  tensor([[54.3555, 51.5364]], grad_fn=<AddmmBackward0>)
Negative Impact Pruned Model Predictions:  tensor([[63.2352, 60.6522]], grad_fn=<AddmmBackward0>)
Baseline Model Features shape  torch.Size([1, 4096])
Noisy Pruned Model Features shape  torch.Size([1, 4096])
Negative Impact Pruned Model Features shape  torch.Size([1, 4096])


# Next steps

- Getting the activations of each channel in the last convolutional layer for your images 
- Weighting each channel's activation map by its computed importance score
- Aggregating these weighted activations to form a final heatmap
- Visualizing this heatmap overlaid on the original image

# 1) Sort the importance scores by channel ID and normalize them to sum to 1

In [ ]:
#set numpy print options
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
importance_scores = np.load('Ranking_arrays/importance_scores.npy')
importance_scores_sorted_by_index = importance_scores[importance_scores[:,0].argsort()][:,1]

# Normalize the importance scores to sum to 1
normalized_importance_scores_sorted_by_index = importance_scores_sorted_by_index / np.sum(importance_scores_sorted_by_index)


# 2) Extracting the activations of the last convolutional layer for the images in the dataset

In [45]:

class FeatureMapHook:
    """Hook to extract feature maps from neural network layers."""
    
    def __init__(self):
        self.feature_maps = []
    
    def __call__(self, module, input, output):
        # Detach from computation graph and move to CPU
        self.feature_maps.append(output.detach().cpu())

def get_feature_maps(model, dataloader, layer_name, device):
    """
    Extracts the feature maps of a specific layer from a model.
    
    Args:
        model (nn.Module): The neural network model.
        dataloader (DataLoader): DataLoader for evaluation.
        layer_name (str): The name of the layer to extract feature maps from.
        device (str): Device to run the model on ('cuda' or 'cpu').
        
    Returns:
        np.ndarray: The feature maps as a numpy array with shape (240, num_features).
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)
    
    # Register a hook to extract feature maps
    hook = FeatureMapHook()
    target_layer = dict(model.named_modules())[layer_name]
    hook_handle = target_layer.register_forward_hook(hook)
    
    # Forward pass to extract feature maps from the dataloader
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            model(inputs)

    # Remove the hook
    hook_handle.remove()
    
    # Process the feature maps to get the desired shape
    all_features = []
    
    for batch_features in hook.feature_maps:
        # Add batch features to our collection
        all_features.append(batch_features)
    
    # Concatenate all batches and convert to numpy
    features_tensor = torch.cat(all_features, dim=0)
    
    # Ensure we have exactly the number of samples we expect in the dataloader 
    assert features_tensor.shape[0] == len(dataloader.dataset) 
    
    # Convert to numpy array
    features_array = features_tensor.numpy()
    
    return features_array


layer_28_feature_maps_base = get_feature_maps(baseline_model, dataloaders['val'], 'features.28', device)
layer_28_feature_maps_noisy = get_feature_maps(noisy_pruned_model, dataloaders['val'], 'features.28', device)
layer_28_feature_maps_neg_impact = get_feature_maps(negative_impact_pruned_model, dataloaders['val'], 'features.28', device)

# check shapes
print("Layer 28 feature maps shape (Baseline): ", layer_28_feature_maps_base.shape)
print("Layer 28 feature maps shape (Noisy Pruned): ", layer_28_feature_maps_noisy.shape)
print("Layer 28 feature maps shape (Negative Impact Pruned): ", layer_28_feature_maps_neg_impact.shape)



Layer 28 feature maps shape (Baseline):  (480, 512, 14, 14)
Layer 28 feature maps shape (Noisy Pruned):  (480, 512, 14, 14)
Layer 28 feature maps shape (Negative Impact Pruned):  (480, 512, 14, 14)


In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm
import pathlib
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms


def compute_heatmap(feature_maps, importance_scores):
    """
    Compute a weighted heatmap using feature maps and importance scores.
    
    Args:
        feature_maps: Feature maps from the last conv layer, shape (512, H, W)
        importance_scores: Importance score for each channel, shape (512,)
        
    Returns:
        A weighted heatmap
    """
    # Ensure importance_scores has the right shape for broadcasting (512, 1, 1)
    importance_scores = importance_scores.reshape(-1, 1, 1)
    
    # Weight each feature map by its importance score
    weighted_maps = feature_maps * importance_scores
    
    # Sum along the channel dimension to get the final heatmap
    heatmap = np.sum(weighted_maps, axis=0)
    
    return heatmap


def overlay_mask(img, mask, alpha=0.6, colormap='jet'):
    """
    Overlay a heatmap mask on an image.
    
    Args:
        img: Original image (PIL Image)
        mask: Heatmap mask (numpy array)
        alpha: Transparency of the overlay (lower = more transparent)
        colormap: Colormap name
        
    Returns:
        Overlaid image as PIL Image
    """
    # Convert PIL image to numpy array
    img_array = np.array(img)
    
    # Normalize the mask
    if np.max(mask) > 0:
        mask = mask / np.max(mask)
    
    # Resize mask to match image dimensions
    mask_resized = cv2.resize(mask, (img.width, img.height))
    
    # Apply colormap
    # Use plt.cm instead of cm.get_cmap to avoid deprecation warning
    cmap = plt.cm.get_cmap(colormap)
    heatmap_colored = cmap(mask_resized)
    # Convert to RGB (remove alpha channel) and scale to 0-255
    heatmap_colored = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    
    # Blend original image with heatmap
    blended = cv2.addWeighted(img_array, alpha, heatmap_colored, 1-alpha, 0)
    
    # Convert back to PIL Image
    return Image.fromarray(blended)


def create_heatmaps_from_dataloader(model, dataloader, channel_importance_scores, output_dir, device='cuda',
                                   feature_layer_name='features.28', batch_limit=None):
    """
    Create and save heatmaps for images in a dataloader using channel importance scores.
    
    Args:
        model: The model to extract feature maps
        dataloader: PyTorch DataLoader containing images
        channel_importance_scores: Array of importance scores for each channel (512,)
        output_dir: Directory to save the output heatmaps
        device: Device to run model on ('cuda' or 'cpu')
        feature_layer_name: Name of the layer to extract features from
        batch_limit: Maximum number of batches to process (None = all)
    """
    # Create output directory if it doesn't exist
    output_dir = pathlib.Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Set model to evaluation mode
    model.eval()
    model.to(device)
    
    # Define hook to capture feature maps
    activation = {}
    def hook_fn(module, input, output):
        activation['features'] = output.detach().cpu().numpy()
    
    # Register hook to get feature maps from the specified layer
    for name, module in model.named_modules():
        if name == feature_layer_name:
            hook_handle = module.register_forward_hook(hook_fn)
            break
    else:
        raise ValueError(f"Layer {feature_layer_name} not found in the model")
    
    # Define inverse normalization to recover original images
    inv_normalize = transforms.Compose([
        transforms.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
    ])
    
    # Process each batch
    heatmap_collection = []
    img_idx = 0
    
    try:
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(dataloader):
                if batch_limit is not None and batch_idx >= batch_limit:
                    break
                    
                # Move inputs to device
                inputs = inputs.to(device)
                
                # Forward pass through the model
                _ = model(inputs)
                
                # Get feature maps from hook
                batch_feature_maps = activation['features']
                
                # Process each image in the batch
                for i in range(inputs.size(0)):
                    # Get feature maps for this image
                    feature_maps = batch_feature_maps[i]  # Shape: (512, H, W)
                    
                    # Compute heatmap
                    heatmap = compute_heatmap(feature_maps, channel_importance_scores)
                    heatmap_collection.append(heatmap)
                    
                    # Convert tensor back to image
                    img_tensor = inputs[i].cpu()
                    # Denormalize the image
                    img_tensor = inv_normalize(img_tensor)
                    img_tensor = torch.clamp(img_tensor, 0, 1)
                    img_np = img_tensor.permute(1, 2, 0).numpy() * 255
                    original_img = Image.fromarray(img_np.astype(np.uint8))
                    
                    # Overlay heatmap on original image
                    overlaid_img = overlay_mask(original_img, heatmap)
                    
                    # Get class label info if available
                    if hasattr(labels[i], 'item'):
                        if labels[i].dim() > 0:
                            class_info = f"_class{labels[i][0]:.1f}_{labels[i][1]:.1f}"
                        else:
                            class_info = f"_class{labels[i].item()}"
                    else:
                        class_info = ""
                    
                    # Save the overlaid image
                    output_path = output_dir / f"heatmap_idx{img_idx}{class_info}.png"
                    overlaid_img.save(output_path)
                    
                    print(f"Processed image {img_idx}")
                    img_idx += 1
                
                print(f"Completed batch {batch_idx+1}/{len(dataloader)}")
    
    finally:
        # Always remove the hook
        hook_handle.remove()
    
    # Save the heatmap collection for later use
    np.save(f"{output_dir}/heatmap_collection.npy", np.array(heatmap_collection))
    
    return heatmap_collection

In [ ]:
# Get image paths from validation dataset
model = baseline_model
image_paths = [dataset.data.iloc[idx, 3] for idx in val_dataset.indices]
output_dir = pathlib.Path('Heatmaps')
features = layer_28_feature_maps_base
channel_importance_scores = normalized_importance_scores_sorted_by_index

# Create heatmaps for the validation set
create_heatmaps_from_dataloader(model, val_dataloader, channel_importance_scores, output_dir, device=device, feature_layer_name='features.28', batch_limit=None)

(480, 512, 14, 14)
(512,)


/tmp/ipykernel_337739/277430050.py:50: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(colormap)


ValueError: operands could not be broadcast together with shapes (512,512,3) (512,512,3,4) 